In [43]:
#Import
import os
import pickle
from datetime import datetime

import keras.backend as K
import numpy as np
from keras.engine.topology import Container
from keras.optimizers import Adam

from config import BEGANConfig
from PIL import Image

#from generate_image import generate
#from began.models import build_model, load_model_weight

from keras import backend as K
from keras.engine import Input, Model
from keras.engine.topology import Container, Layer
from keras.layers import Lambda, Concatenate, Flatten, Dense, Reshape, Convolution2D, UpSampling2D
import h5py

In [44]:
#Training
ESC = chr(0x1b)
UP = ESC+"[1A"

def training(config: BEGANConfig, epochs=3):
    # loading dataset, and let values to 0.0 ~ 1.0
#    with open(config.dataset_filename, 'rb') as f:
#        dataset = pickle.load(f)  # type: np.ndarray
#        dataset = dataset / 255.0
#        info(dataset.shape)

    with h5py.File("../../../CelebA_64_data.h5", "r") as hf:
        dataset = hf["data"][:10000].astype(np.float32)
        dataset = dataset/255.0
        info(dataset.shape)
        dataset = dataset.transpose(0, 2, 3, 1)
        info(dataset.shape)

    batch_size = config.batch_size

    # building model and loading weight(if exists)
    autoencoder, generator, discriminator = build_model(config)
    load_model_weight(autoencoder, config.autoencoder_weight_filename)
    load_model_weight(generator, config.generator_weight_filename)
    load_model_weight(discriminator, config.discriminator_weight_filename)

    loss_d = DiscriminatorLoss(config.initial_k)  # special? loss object for BEGAN
    discriminator.compile(optimizer=Adam(), loss=loss_d)
    generator.compile(optimizer=Adam(), loss=create_generator_loss(autoencoder))
    lr_decay_step = 0
    last_m_global = np.Inf
    log_recorder = LogRecorder(config.training_log)

    # print("Generator Update Variables")
    # print_model_updates(generator)
    #
    # print("Discriminator Update Variables")
    # print_model_updates(discriminator)

    for ep in range(1, epochs+1):
        np.random.seed(ep * 100)
        # generate Z layer values for discriminator and generator
        zd = np.random.uniform(-1, 1, (len(dataset), config.hidden_size))
        zg = np.random.uniform(-1, 1, (len(dataset), config.hidden_size))

        # shuffle dataset index
        index_order = np.arange(len(dataset))
        np.random.shuffle(index_order)

        # set Learning Rate
        lr = max(config.initial_lr * (config.lr_decay_rate ** lr_decay_step), config.min_lr)
        K.set_value(generator.optimizer.lr, lr)
        K.set_value(discriminator.optimizer.lr, lr)
        m_global_history = []
        info("LearningRate=%.7f" % lr)
        batch_len = len(dataset)//batch_size

        for b_idx in range(batch_len):
            index_list = index_order[b_idx*batch_size:(b_idx+1)*batch_size]

            # training discriminator
            in_x1 = dataset[index_list]  # (bs, row, col, ch)
            in_x2 = generator.predict_on_batch(zd[index_list])
            in_x = np.concatenate([in_x1, in_x2], axis=-1)  # (bs, row, col, ch*2)
            loss_discriminator = discriminator.train_on_batch(in_x, in_x)

            # training generator
            in_x1 = zg[index_list]
            loss_generator = generator.train_on_batch(in_x1, np.zeros_like(in_x2))  # y_true is meaningless

            # record M-Global
            m_global_history.append(loss_d.m_global)
            if b_idx > 0:
                print(UP + UP)
            log_info = dict(
                epoch=ep,
                batch_index=b_idx,
                batch_len=batch_len,
                m_global=loss_d.m_global,
                loss_discriminator=loss_discriminator,
                loss_generator=loss_generator,
                loss_real_x=loss_d.loss_real_x,
                loss_gen_x=loss_d.loss_gen_x,
                k=loss_d.k,
                lr=lr,
            )
            info("ep=%(epoch)s, b_idx=%(batch_index)s/%(batch_len)s, MGlobal=%(m_global).5f, "
                 "Loss(D)=%(loss_discriminator).5f, Loss(G)=%(loss_generator).5f, Loss(X)=%(loss_real_x).5f, "
                 "Loss(G(Zd))=%(loss_gen_x).5f, K=%(k).6f" % log_info)
            log_recorder.write(**log_info)

        m_global = np.average(m_global_history)
        if last_m_global <= m_global:  # decay LearningRate
            lr_decay_step += 1
        last_m_global = m_global

        # Save Model Weight in each epoch
        autoencoder.save_weights(config.autoencoder_weight_filename)
        generator.save_weights(config.generator_weight_filename)
        discriminator.save_weights(config.discriminator_weight_filename)

        # Generate Image in each epoch for fun
        generate(config, "ep%03d" % ep, generator)


def create_generator_loss(autoencoder: Container):
    def generator_loss(y_true, y_pred):
        y_pred_dash = autoencoder(y_pred)
        return K.mean(K.abs(y_pred - y_pred_dash), axis=[1, 2, 3])
    return generator_loss


class DiscriminatorLoss:
    __name__ = 'discriminator_loss'

    def __init__(self, initial_k=0, lambda_k=0.001, gamma=0.5):
        self.lambda_k = lambda_k
        self.gamma = gamma
        self.k_var = K.variable(initial_k, dtype=K.floatx(), name="discriminator_k")
        self.m_global_var = K.variable(0, dtype=K.floatx(), name="m_global")
        self.loss_real_x_var = K.variable(0, name="loss_real_x")  # for observation
        self.loss_gen_x_var = K.variable(0, name="loss_gen_x")    # for observation
        self.updates = []

    def __call__(self, y_true, y_pred):  # y_true, y_pred shape: (BS, row, col, ch * 2)
        data_true, generator_true = y_true[:, :, :, 0:3], y_true[:, :, :, 3:6]
        data_pred, generator_pred = y_pred[:, :, :, 0:3], y_pred[:, :, :, 3:6]
        loss_data = K.mean(K.abs(data_true - data_pred), axis=[1, 2, 3])
        loss_generator = K.mean(K.abs(generator_true - generator_pred), axis=[1, 2, 3])
        ret = loss_data - self.k_var * loss_generator

        # for updating values in each epoch, use `updates` mechanism
        # DiscriminatorModel collects Loss Function's updates attributes
        mean_loss_data = K.mean(loss_data)
        mean_loss_gen = K.mean(loss_generator)

        # update K
        new_k = self.k_var + self.lambda_k * (self.gamma * mean_loss_data - mean_loss_gen)
        new_k = K.clip(new_k, 0, 1)
        self.updates.append(K.update(self.k_var, new_k))

        # calculate M-Global
        m_global = mean_loss_data + K.abs(self.gamma * mean_loss_data - mean_loss_gen)
        self.updates.append(K.update(self.m_global_var, m_global))

        # let loss_real_x mean_loss_data
        self.updates.append(K.update(self.loss_real_x_var, mean_loss_data))

        # let loss_gen_x mean_loss_gen
        self.updates.append(K.update(self.loss_gen_x_var, mean_loss_gen))

        return ret

    @property
    def k(self):
        return K.get_value(self.k_var)

    @property
    def m_global(self):
        return K.get_value(self.m_global_var)

    @property
    def loss_real_x(self):
        return K.get_value(self.loss_real_x_var)

    @property
    def loss_gen_x(self):
        return K.get_value(self.loss_gen_x_var)


def info(msg):
    now = datetime.now()
    print("%s: %s" % (now, msg))


class LogRecorder:
    def __init__(self, log_filename):
        if not os.path.exists(os.path.dirname(log_filename)):
            os.makedirs((os.path.dirname(log_filename)))
        self.file_out = open(log_filename, "wt")
        self.columns = None

    def write(self, **kwargs):
        if not self.columns:
            self.columns = list(sorted(kwargs.keys()))
            self.file_out.write(",".join(self.columns) + "\n")
        values = [str(kwargs.get(x, "")) for x in self.columns]
        self.file_out.write(",".join(values) + "\n")
        self.file_out.flush()


def print_model_updates(model):
    training_updates = model.optimizer.get_updates(
        model._collected_trainable_weights,
        model.constraints,
        model.total_loss)
    updates = model.updates + training_updates
    print("\n".join(sorted([str(x[0]) for x in updates])))

In [45]:
#Generate image
def generate(config: BEGANConfig, dir_name, generator=None):
    output_dir = "%s/%s" % (config.generated_dir, dir_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if generator is None:
        generator = build_generator(config)
        load_model_weight(generator, config.generator_weight_filename)

    np.random.seed(999)
    num_image = 20
    z = np.random.uniform(-1, 1, config.hidden_size * num_image).reshape((num_image, config.hidden_size))
    x = generator.predict(z)
    images = np.clip(x * 255, 0, 255).astype("uint8")
    print((x.shape, np.min(x), np.max(x)))

    for i in range(num_image):
        image_x = images[i]
        img = Image.fromarray(image_x)
        img.save("%s/gen_%03d.jpg" % (output_dir, i))



In [ ]:
#Main
def build_model(config: BEGANConfig):
    K.set_image_data_format('channels_last')

    autoencoder = build_autoencoder(config)
    generator = build_generator(config)
    discriminator = build_discriminator(config, autoencoder)

    return autoencoder, generator, discriminator


def load_model_weight(model: Container, weight_file):
    if os.path.exists(weight_file):
        print("loading model weight: %s" % weight_file)
        model.load_weights(weight_file)


def build_autoencoder(config: BEGANConfig, name="autoencoder"):
    encoder = build_encoder(config, name="%s/encoder" % name)
    decoder = build_decoder(config, name="%s/decoder" % name)
    autoencoder = Container(encoder.inputs, decoder(encoder.outputs), name=name)
    return autoencoder


def build_generator(config: BEGANConfig):
    decoder = build_decoder(config, name="generator_decoder")
    generator = Model(decoder.inputs, decoder.outputs, name="generator")
    return generator


def build_encoder(config: BEGANConfig, name="encoder"):
    n_filters = config.n_filters
    hidden_size = config.hidden_size
    n_layer = config.n_layer_in_conv

    dx = image_input = Input((config.image_height, config.image_width, 3))

    # output: (N, 32, 32, n_filters)
    dx = convolution_image_for_encoding(dx, n_filters, strides=(2, 2), name="%s/L1" % name, n_layer=n_layer)

    # output: (N, 16, 16, n_filters*2)
    dx = convolution_image_for_encoding(dx, n_filters * 2, strides=(2, 2), name="%s/L2" % name, n_layer=n_layer)

    # output: (N, 8, 8, n_filters*3)
    dx = convolution_image_for_encoding(dx, n_filters * 3, strides=(2, 2), name="%s/L3" % name, n_layer=n_layer)

    # output: (N, 8, 8, n_filters*4)
    dx = convolution_image_for_encoding(dx, n_filters * 4, strides=(1, 1), name="%s/L4" % name, n_layer=n_layer)

    dx = Flatten()(dx)
    hidden = Dense(hidden_size, activation='linear', name="%s/Dense" % name)(dx)

    encoder = Container(image_input, hidden, name=name)
    return encoder


def build_decoder(config: BEGANConfig, name):
    """
    generator and decoder( of discriminator) have same network structure, but don't share weights.
    This function takes different input layer, flow another network, and return different output layer.
    """
    n_filters = config.n_filters
    n_layer = config.n_layer_in_conv

    dx = input_z = Input((64, ))
    dx = Dense((8*8*n_filters), activation='linear', name="%s/Dense" % name)(dx)
    dx = Reshape((8, 8, n_filters))(dx)

    # output: (N, 16, 16, n_filters)
    dx = convolution_image_for_decoding(dx, n_filters, upsample=True, name="%s/L1" % name, n_layer=n_layer)

    # output: (N, 32, 32, n_filters)
    dx = convolution_image_for_decoding(dx, n_filters, upsample=True, name="%s/L2" % name, n_layer=n_layer)

    # output: (N, 64, 64, n_filters)
    dx = convolution_image_for_decoding(dx, n_filters, upsample=True, name="%s/L3" % name, n_layer=n_layer)

    # output: (N, 64, 64, n_filters)
    dx = convolution_image_for_decoding(dx, n_filters, upsample=False, name="%s/L4" % name, n_layer=n_layer)

    # output: (N, 64, 64, 3)
    image_output = Convolution2D(3, (3, 3), padding="same", activation="linear", name="%s/FinalConv" % name)(dx)
    decoder = Container(input_z, image_output, name=name)
    return decoder


def build_discriminator(config: BEGANConfig, autoencoder: Container):
    """
    
    Keras Model class is able to have several inputs/outputs. 
    But, loss functions should be defined each other, and the loss function cannot reference other inputs/outputs.
    For computing loss, two inputs/outputs are concatenated.
    """
    # IN Shape: [ImageHeight, ImageWidth, (real data(3 channels) + generated data(3 channels))]
    in_out_shape = (config.image_height, config.image_width, 3 * 2)
    all_input = Input(in_out_shape)

    # Split Input Data
    data_input = Lambda(lambda x: x[:, :, :, 0:3], output_shape=(config.image_height, config.image_width, 3))(all_input)
    generator_input = Lambda(lambda x: x[:, :, :, 3:6], output_shape=(config.image_height, config.image_width, 3))(all_input)

    # use same autoencoder(weights are shared)
    data_output = autoencoder(data_input)  # (bs, row, col, ch)
    generator_output = autoencoder(generator_input)

    # concatenate output to be same shape of input
    all_output = Concatenate(axis=-1)([data_output, generator_output])

    discriminator = DiscriminatorModel(all_input, all_output, name="discriminator")
    return discriminator


def convolution_image_for_encoding(x, filters, strides=(1, 1), name=None, n_layer=2):
    for i in range(1, n_layer):
        x = Convolution2D(filters, (3, 3), activation="elu", padding="same", name="%s/Conv%d" % (name, i))(x)

    x = Convolution2D(filters, (3, 3), activation="elu", padding="same", strides=strides,
                      name="%s/Conv%d" % (name, n_layer))(x)
    return x


def convolution_image_for_decoding(x, filters, upsample=None, name=None, n_layer=2):
    for i in range(1, n_layer+1):
        x = Convolution2D(filters, (3, 3), activation="elu", padding="same", name="%s/Conv%d" % (name, i))(x)
    if upsample:
        x = UpSampling2D()(x)
    return x


class DiscriminatorModel(Model):
    """Model which collects updates from loss_func.updates"""

    @property
    def updates(self):
        updates = super().updates
        if hasattr(self, 'loss_functions'):
            for loss_func in self.loss_functions:
                if hasattr(loss_func, 'updates'):
                    updates += loss_func.updates
        return updates

In [ ]:
config = BEGANConfig()
training(config, epochs=500)

2018-06-16 07:12:26.854022: (10000, 3, 64, 64)
2018-06-16 07:12:26.854203: (10000, 64, 64, 3)
loading model weight: /content/clouderizer/fast.ai/fastai/keras_BEGAN/src/began/../../resource/autoencoder.hd5
loading model weight: /content/clouderizer/fast.ai/fastai/keras_BEGAN/src/began/../../resource/generator.hd5
loading model weight: /content/clouderizer/fast.ai/fastai/keras_BEGAN/src/began/../../resource/discriminator.hd5
2018-06-16 07:12:30.670215: LearningRate=0.0001000
2018-06-16 07:12:37.960937: ep=1, b_idx=0/625, MGlobal=0.09927, Loss(D)=0.07572, Loss(G)=0.03741, Loss(X)=0.07572, Loss(G(Zd))=0.01431, K=0.000024

2018-06-16 07:12:38.480684: ep=1, b_idx=1/625, MGlobal=0.20913, Loss(D)=0.15632, Loss(G)=0.04298, Loss(X)=0.15632, Loss(G(Zd))=0.02535, K=0.000076

2018-06-16 07:12:38.946908: ep=1, b_idx=2/625, MGlobal=0.21527, Loss(D)=0.16662, Loss(G)=0.04461, Loss(X)=0.16663, Loss(G(Zd))=0.03466, K=0.000125

2018-06-16 07:12:39.430740: ep=1, b_idx=3/625, MGlobal=0.11849, Loss(D)=0.1060


2018-06-16 07:13:01.779234: ep=1, b_idx=50/625, MGlobal=0.10928, Loss(D)=0.08480, Loss(G)=0.01689, Loss(X)=0.08482, Loss(G(Zd))=0.01795, K=0.001132

2018-06-16 07:13:02.256742: ep=1, b_idx=51/625, MGlobal=0.10714, Loss(D)=0.08243, Loss(G)=0.01627, Loss(X)=0.08245, Loss(G(Zd))=0.01653, K=0.001157

2018-06-16 07:13:02.731651: ep=1, b_idx=52/625, MGlobal=0.10729, Loss(D)=0.08227, Loss(G)=0.01689, Loss(X)=0.08229, Loss(G(Zd))=0.01615, K=0.001182

2018-06-16 07:13:03.204212: ep=1, b_idx=53/625, MGlobal=0.10299, Loss(D)=0.07964, Loss(G)=0.01772, Loss(X)=0.07965, Loss(G(Zd))=0.01649, K=0.001205

2018-06-16 07:13:03.679948: ep=1, b_idx=54/625, MGlobal=0.10654, Loss(D)=0.08269, Loss(G)=0.01863, Loss(X)=0.08271, Loss(G(Zd))=0.01753, K=0.001229

2018-06-16 07:13:04.163196: ep=1, b_idx=55/625, MGlobal=0.10517, Loss(D)=0.08200, Loss(G)=0.02059, Loss(X)=0.08202, Loss(G(Zd))=0.01786, K=0.001252

2018-06-16 07:13:04.635542: ep=1, b_idx=56/625, MGlobal=0.11212, Loss(D)=0.08793, Loss(G)=0.01838, Loss(X


2018-06-16 07:13:27.287802: ep=1, b_idx=103/625, MGlobal=0.09725, Loss(D)=0.07672, Loss(G)=0.01615, Loss(X)=0.07677, Loss(G(Zd))=0.01790, K=0.002477

2018-06-16 07:13:27.787301: ep=1, b_idx=104/625, MGlobal=0.10662, Loss(D)=0.08171, Loss(G)=0.01405, Loss(X)=0.08175, Loss(G(Zd))=0.01601, K=0.002501

2018-06-16 07:13:28.257215: ep=1, b_idx=105/625, MGlobal=0.10633, Loss(D)=0.07986, Loss(G)=0.01477, Loss(X)=0.07989, Loss(G(Zd))=0.01351, K=0.002528

2018-06-16 07:13:28.729694: ep=1, b_idx=106/625, MGlobal=0.10439, Loss(D)=0.07877, Loss(G)=0.01240, Loss(X)=0.07880, Loss(G(Zd))=0.01381, K=0.002553

2018-06-16 07:13:29.242033: ep=1, b_idx=107/625, MGlobal=0.11323, Loss(D)=0.08313, Loss(G)=0.01273, Loss(X)=0.08315, Loss(G(Zd))=0.01151, K=0.002584

2018-06-16 07:13:29.723663: ep=1, b_idx=108/625, MGlobal=0.11144, Loss(D)=0.08250, Loss(G)=0.01217, Loss(X)=0.08253, Loss(G(Zd))=0.01236, K=0.002612

2018-06-16 07:13:30.215215: ep=1, b_idx=109/625, MGlobal=0.10097, Loss(D)=0.07528, Loss(G)=0.01133,


2018-06-16 07:13:52.670013: ep=1, b_idx=155/625, MGlobal=0.10392, Loss(D)=0.07663, Loss(G)=0.01233, Loss(X)=0.07668, Loss(G(Zd))=0.01109, K=0.003800

2018-06-16 07:13:53.159496: ep=1, b_idx=156/625, MGlobal=0.12042, Loss(D)=0.08867, Loss(G)=0.01265, Loss(X)=0.08871, Loss(G(Zd))=0.01265, K=0.003832

2018-06-16 07:13:53.637928: ep=1, b_idx=157/625, MGlobal=0.10688, Loss(D)=0.07962, Loss(G)=0.01336, Loss(X)=0.07967, Loss(G(Zd))=0.01262, K=0.003859

2018-06-16 07:13:54.123825: ep=1, b_idx=158/625, MGlobal=0.10927, Loss(D)=0.08144, Loss(G)=0.01252, Loss(X)=0.08149, Loss(G(Zd))=0.01296, K=0.003887

2018-06-16 07:13:54.602194: ep=1, b_idx=159/625, MGlobal=0.10753, Loss(D)=0.07963, Loss(G)=0.01112, Loss(X)=0.07967, Loss(G(Zd))=0.01198, K=0.003915

2018-06-16 07:13:55.075655: ep=1, b_idx=160/625, MGlobal=0.11027, Loss(D)=0.08054, Loss(G)=0.01246, Loss(X)=0.08059, Loss(G(Zd))=0.01060, K=0.003944

2018-06-16 07:13:55.556532: ep=1, b_idx=161/625, MGlobal=0.10724, Loss(D)=0.07987, Loss(G)=0.01039,


2018-06-16 07:14:17.515862: ep=1, b_idx=207/625, MGlobal=0.09371, Loss(D)=0.08241, Loss(G)=0.02524, Loss(X)=0.08257, Loss(G(Zd))=0.03014, K=0.005108

2018-06-16 07:14:17.988256: ep=1, b_idx=208/625, MGlobal=0.08748, Loss(D)=0.07475, Loss(G)=0.01790, Loss(X)=0.07488, Loss(G(Zd))=0.02484, K=0.005121

2018-06-16 07:14:18.460557: ep=1, b_idx=209/625, MGlobal=0.10534, Loss(D)=0.08104, Loss(G)=0.01619, Loss(X)=0.08113, Loss(G(Zd))=0.01635, K=0.005145

2018-06-16 07:14:18.940304: ep=1, b_idx=210/625, MGlobal=0.12161, Loss(D)=0.09083, Loss(G)=0.02434, Loss(X)=0.09090, Loss(G(Zd))=0.01474, K=0.005176

2018-06-16 07:14:19.415088: ep=1, b_idx=211/625, MGlobal=0.11030, Loss(D)=0.08975, Loss(G)=0.01788, Loss(X)=0.08987, Loss(G(Zd))=0.02451, K=0.005196

2018-06-16 07:14:19.898731: ep=1, b_idx=212/625, MGlobal=0.11040, Loss(D)=0.08515, Loss(G)=0.01185, Loss(X)=0.08524, Loss(G(Zd))=0.01747, K=0.005221

2018-06-16 07:14:20.384991: ep=1, b_idx=213/625, MGlobal=0.11863, Loss(D)=0.08727, Loss(G)=0.01161,

...


2018-06-16 07:25:33.936161: ep=3, b_idx=363/625, MGlobal=0.07449, Loss(D)=0.06457, Loss(G)=0.02249, Loss(X)=0.06532, Loss(G(Zd))=0.02349, K=0.031842

2018-06-16 07:25:34.420903: ep=3, b_idx=364/625, MGlobal=0.09355, Loss(D)=0.07624, Loss(G)=0.02682, Loss(X)=0.07694, Loss(G(Zd))=0.02186, K=0.031859

2018-06-16 07:25:34.896027: ep=3, b_idx=365/625, MGlobal=0.09481, Loss(D)=0.08012, Loss(G)=0.02976, Loss(X)=0.08097, Loss(G(Zd))=0.02664, K=0.031873

2018-06-16 07:25:35.371989: ep=3, b_idx=366/625, MGlobal=0.07552, Loss(D)=0.06861, Loss(G)=0.02177, Loss(X)=0.06953, Loss(G(Zd))=0.02878, K=0.031879

2018-06-16 07:25:35.855876: ep=3, b_idx=367/625, MGlobal=0.08898, Loss(D)=0.07262, Loss(G)=0.01880, Loss(X)=0.07329, Loss(G(Zd))=0.02095, K=0.031894

2018-06-16 07:25:36.333049: ep=3, b_idx=368/625, MGlobal=0.08812, Loss(D)=0.07017, Loss(G)=0.01675, Loss(X)=0.07075, Loss(G(Zd))=0.01800, K=0.031912

2018-06-16 07:25:36.803562: ep=3, b_idx=369/625, MGlobal=0.09084, Loss(D)=0.07059, Loss(G)=0.02071,


2018-06-16 07:25:59.081960: ep=3, b_idx=415/625, MGlobal=0.07590, Loss(D)=0.06536, Loss(G)=0.01802, Loss(X)=0.06612, Loss(G(Zd))=0.02328, K=0.032497

2018-06-16 07:25:59.565654: ep=3, b_idx=416/625, MGlobal=0.08577, Loss(D)=0.06839, Loss(G)=0.01719, Loss(X)=0.06896, Loss(G(Zd))=0.01768, K=0.032514

2018-06-16 07:26:00.061001: ep=3, b_idx=417/625, MGlobal=0.08758, Loss(D)=0.06941, Loss(G)=0.02054, Loss(X)=0.06998, Loss(G(Zd))=0.01738, K=0.032532

2018-06-16 07:26:00.545933: ep=3, b_idx=418/625, MGlobal=0.08686, Loss(D)=0.07043, Loss(G)=0.02357, Loss(X)=0.07107, Loss(G(Zd))=0.01974, K=0.032547

2018-06-16 07:26:01.025396: ep=3, b_idx=419/625, MGlobal=0.08310, Loss(D)=0.06973, Loss(G)=0.02929, Loss(X)=0.07047, Loss(G(Zd))=0.02260, K=0.032560

2018-06-16 07:26:01.535705: ep=3, b_idx=420/625, MGlobal=0.07880, Loss(D)=0.07024, Loss(G)=0.02375, Loss(X)=0.07115, Loss(G(Zd))=0.02793, K=0.032568

2018-06-16 07:26:02.026060: ep=3, b_idx=421/625, MGlobal=0.09652, Loss(D)=0.07835, Loss(G)=0.01980,


2018-06-16 07:26:24.273062: ep=3, b_idx=467/625, MGlobal=0.08426, Loss(D)=0.06760, Loss(G)=0.02193, Loss(X)=0.06820, Loss(G(Zd))=0.01804, K=0.033128

2018-06-16 07:26:24.752858: ep=3, b_idx=468/625, MGlobal=0.08958, Loss(D)=0.07324, Loss(G)=0.02357, Loss(X)=0.07395, Loss(G(Zd))=0.02135, K=0.033143

2018-06-16 07:26:25.227437: ep=3, b_idx=469/625, MGlobal=0.07845, Loss(D)=0.06634, Loss(G)=0.02663, Loss(X)=0.06707, Loss(G(Zd))=0.02215, K=0.033155

2018-06-16 07:26:25.701026: ep=3, b_idx=470/625, MGlobal=0.08100, Loss(D)=0.07041, Loss(G)=0.02324, Loss(X)=0.07127, Loss(G(Zd))=0.02590, K=0.033164

2018-06-16 07:26:26.184247: ep=3, b_idx=471/625, MGlobal=0.08527, Loss(D)=0.07139, Loss(G)=0.02174, Loss(X)=0.07215, Loss(G(Zd))=0.02296, K=0.033178

2018-06-16 07:26:26.673040: ep=3, b_idx=472/625, MGlobal=0.07780, Loss(D)=0.06510, Loss(G)=0.02202, Loss(X)=0.06579, Loss(G(Zd))=0.02089, K=0.033190

2018-06-16 07:26:27.150055: ep=3, b_idx=473/625, MGlobal=0.08148, Loss(D)=0.06788, Loss(G)=0.02302,


2018-06-16 07:26:49.225282: ep=3, b_idx=519/625, MGlobal=0.08444, Loss(D)=0.06939, Loss(G)=0.02568, Loss(X)=0.07009, Loss(G(Zd))=0.02069, K=0.033763

2018-06-16 07:26:49.699110: ep=3, b_idx=520/625, MGlobal=0.08117, Loss(D)=0.06937, Loss(G)=0.02602, Loss(X)=0.07019, Loss(G(Zd))=0.02411, K=0.033774

2018-06-16 07:26:50.172516: ep=3, b_idx=521/625, MGlobal=0.08297, Loss(D)=0.07115, Loss(G)=0.02608, Loss(X)=0.07199, Loss(G(Zd))=0.02502, K=0.033785

2018-06-16 07:26:50.648078: ep=3, b_idx=522/625, MGlobal=0.08410, Loss(D)=0.07249, Loss(G)=0.02084, Loss(X)=0.07336, Loss(G(Zd))=0.02594, K=0.033796

2018-06-16 07:26:51.123891: ep=3, b_idx=523/625, MGlobal=0.07816, Loss(D)=0.06499, Loss(G)=0.02041, Loss(X)=0.06568, Loss(G(Zd))=0.02036, K=0.033808

2018-06-16 07:26:51.602266: ep=3, b_idx=524/625, MGlobal=0.08627, Loss(D)=0.07022, Loss(G)=0.02281, Loss(X)=0.07090, Loss(G(Zd))=0.02007, K=0.033823

2018-06-16 07:26:52.073967: ep=3, b_idx=525/625, MGlobal=0.07844, Loss(D)=0.06626, Loss(G)=0.02581,


2018-06-16 07:27:14.023448: ep=3, b_idx=571/625, MGlobal=0.07596, Loss(D)=0.06532, Loss(G)=0.02255, Loss(X)=0.06611, Loss(G(Zd))=0.02321, K=0.034400

2018-06-16 07:27:14.501877: ep=3, b_idx=572/625, MGlobal=0.08310, Loss(D)=0.06936, Loss(G)=0.02171, Loss(X)=0.07012, Loss(G(Zd))=0.02208, K=0.034413

2018-06-16 07:27:14.975904: ep=3, b_idx=573/625, MGlobal=0.08587, Loss(D)=0.07058, Loss(G)=0.02138, Loss(X)=0.07130, Loss(G(Zd))=0.02109, K=0.034428

2018-06-16 07:27:15.456652: ep=3, b_idx=574/625, MGlobal=0.08216, Loss(D)=0.06790, Loss(G)=0.02308, Loss(X)=0.06862, Loss(G(Zd))=0.02076, K=0.034441

2018-06-16 07:27:15.935569: ep=3, b_idx=575/625, MGlobal=0.08497, Loss(D)=0.07115, Loss(G)=0.02463, Loss(X)=0.07194, Loss(G(Zd))=0.02295, K=0.034454

2018-06-16 07:27:16.415001: ep=3, b_idx=576/625, MGlobal=0.07771, Loss(D)=0.06698, Loss(G)=0.02526, Loss(X)=0.06781, Loss(G(Zd))=0.02400, K=0.034464

2018-06-16 07:27:16.891838: ep=3, b_idx=577/625, MGlobal=0.08515, Loss(D)=0.07253, Loss(G)=0.02331,